Ayyub Alzahem 3700203

<h3>Unigram Inverted Index

In [62]:
## import the necessary libraries
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt') 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import os
import string
import copy
import pickle 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ayyub\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ayyub\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [66]:
title = "20_newsgroups"
os.chdir("C:/20_newsgroups")

In [67]:
paths = []
for (dirpath, dirnames, filenames) in os.walk(str(os.getcwd())+'/'+title+'/'):
    for i in filenames:
        paths.append(str(dirpath)+str("\\")+i) 

In [68]:
print(dirpath)

C:\20_newsgroups/20_newsgroups/alt.atheism


<h3>Preprocessing </h3>

In [69]:
#Removing stop words 
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words:
            new_text = new_text + " " + w
    return np.char.strip(new_text) 

#Removing punctuation
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, " ", " ")
    data = np.char.replace(data, ',', '')
    return data 

#Convert to lowercase
def convert_lower_case(data):
    return np.char.lower(data)

#Stemming
def stemming(data):
    stemmer= PorterStemmer()

    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return np.char.strip(new_text) 

#Converting numbers to its equivalent words 
def convert_numbers(data):
    data = np.char.replace(data, "0", " zero ")
    data = np.char.replace(data, "1", " one ")
    data = np.char.replace(data, "2", " two ")
    data = np.char.replace(data, "3", " three ")
    data = np.char.replace(data, "4", " four ")
    data = np.char.replace(data, "5", " five ")
    data = np.char.replace(data, "6", " six ")
    data = np.char.replace(data, "7", " seven ")
    data = np.char.replace(data, "8", " eight ")
    data = np.char.replace(data, "9", " nine ")
    return data 

#Removing header 
def remove_header(data):
    try:
        ind = data.index('\n\n')
        data = data[ind:]
    except:
        print("No Header")
    return data 

#Removing apostrophe 
def remove_apostrophe(data):
    return np.char.replace(data, "'", "") 

#Removing single characters 
def remove_single_characters(data):
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if len(w) > 1:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

<h3>Exercise 1: Create a Function preprocess () and invoke the
functions above.  </h3>

In [70]:
def preprocess(data, query):
    if not query:
        data = remove_header(data) 
        data = convert_lower_case(data)
        data = convert_numbers(data)
        data = remove_punctuation(data)
        data = remove_stop_words(data)
        data = remove_apostrophe(data)
        data = remove_single_characters(data)
        data = stemming(data) 
    return data

<h3> Generating Postings for Unigram inverted index 
</h3>

In [80]:
doc = 0
postings = pd.DataFrame()

for path in paths:
    file = open(path, 'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    preprocessed_text = preprocess(text, False)
    
    #Genrate matrex posting list
    if doc%100 == 0:
        print(doc)
    tokens = word_tokenize(str(preprocessed_text))
    for token in tokens:
        if token in postings:
            p = postings[token][0]
            p.add(doc)
            postings[token][0] = p 
        else:
            postings.insert(value=[{doc}], loc=0, column=token)
    doc += 1 

#Save the output:
postings.to_pickle(title + "_unigram_postings")

0


In [81]:
postings

,exam,compil,side,uneven,soc,pub,rutger,ftp,dj,mcdowel,...,nine,decemb,one,modifi,last,alt,resourc,atheism,name,archiv
0,{21},{21},{21},{21},{21},{21},{21},{21},{21},{21},...,"{0, 1, 2, 4, 5, 17, 18, 19}",{0},"{0, 1, 2, 3, 4, 5, 7, 8, 13, 14, 16, 17, 18, 1...","{0, 1}","{0, 1}","{0, 1}",{0},"{0, 1, 5, 6}","{0, 1, 7}","{0, 1}"


<h3> Search Query for Unigram Index

In [82]:
#Read first the stored posting list: 
postings = pd.read_pickle(title + "_unigram_postings")

<h3> Exercise 2: explore posting list. Find how many times the
following words appear in the posting list [‘one’, ‘nine’,
‘exam’] 


In [106]:
s1 = postings['one'][0]
s2 = postings['nine'][0]
s3 = postings['exam'][0]
print(s1)
print(s2)
print(s3)

print('intersection = ', s1 & s2 & s3)

{0, 1, 2, 3, 4, 5, 7, 8, 13, 14, 16, 17, 18, 19, 20, 21}
{0, 1, 2, 4, 5, 17, 18, 19}
{21}
intersection =  {0, 1, 2, 4, 5, 17, 18, 19}


<h3> Exercise 3: What is the posting list of “NOT exam”? 

<h3> Exercise 4. Write code that is able to retrieve a complex
query such as “exam and not resourc.? 
